In [7]:
import pandas as pd
from collections import Counter
import os
KEC_DIR = '/mnt/shared/corpora/German.KEC'
kec = pd.read_table(KEC_DIR + '/kec.csv')
available_speakers = [os.path.splitext(f)[0] for f in os.listdir(KEC_DIR + '/Artikulographie_tsv') if f.endswith('.tsv')]
kec_words = kec[['WordID','Speaker','TimeStartWord', 'TimeEndWord', 'Word']]
kec_words = kec_words.drop_duplicates(subset=['Word'])
kec_words = kec_words.drop(kec_words[kec_words.Word == '<P>'].index) #Pause
kec_words = kec_words[kec_words['Speaker'].str.replace('.TextGrid', '').isin(available_speakers)]

#kec_word_counter= Counter(kec_words['Word'])
#summe = 0
#for word, count in kec_word_counter.most_common(30):
#    summe += count
#    print(f"{word}: {count} {summe}")
#    kec_words.drop(kec_words[kec_words.Word == word].index, inplace=True)

#kec_words.drop(kec_words[(kec_words.TimeEndWord - kec_words.TimeStartWord) < 0.5].index, inplace=True)
#kec_words.drop(kec_words[kec_words['Word'].str.len() < 7].index, inplace=True)




In [ ]:
#erstmal dass was wir eh schon wissen
import fasttext.util
import numpy as np
FASTTEXT_EMBEDDINGS = fasttext.load_model('cc.de.300.bin')
small_kec_df = pd.DataFrame()
small_kec_df['file'] = [kec_words['Speaker'].astype(str).removesuffix('.TextGrid') + kec_words['WordID'].astype(str)]
small_kec_df['label'] = kec_words['Word']
small_kec_df['vector'] = kec_words['Word'].apply(lambda x: np.double(FASTTEXT_EMBEDDINGS.get_word_vector(x)))


In [ ]:
#hier basteln wir uns die signale und nehmen die samplerates gleich mit
import os
import soundfile as sf
import numpy as np

for idx, row in kec_words.iterrows():
    wav_path = os.path.join('/mnt/shared/corpora/German.KEC/Wav_processed_no_names', f"{row['Speaker']}.wav")
    
    if os.path.exists(wav_path):
        data, samplerate = sf.read(wav_path)
        
        start_sample = int(row['TimeStartWord'] * samplerate)
        end_sample = int(row['TimeEndWord'] * samplerate)
        
        small_kec_df['target_sig'] = data[start_sample:end_sample]
        small_kec_df['target_sr'] = samplerate
    else:
        Exception(f"File {wav_path} does not exist")

small_kec_df['len_cp'] = small_kec_df.apply(lambda row: int(2 * np.ceil(44100 / 220 * len(row['Signal']) / row['SampleRate'])), axis=1) 

In [8]:
#jetzt schauen wir uns mal die ema daten an
import pandas as pd
kec_emas = pd.read_table(KEC_DIR + '/Artikulographie_tsv/rec_025_IS_id_049_Art3.tsv', sep='\t')

In [ ]:
import numpy as np
import pandas as pd
from paule import util
import os
import fasttext.util
import math
#fasttext.util.download_model('de', if_exists='ignore')  # English
FASTTEXT_EMBEDDINGS = fasttext.load_model('cc.de.300.bin')
PATH = '/home/bob/AndreWorkland/articubench'
example_tiny = pd.read_pickle(os.path.join(PATH, 'articubench/data/tiny.pkl'))
test_small = pd.read_pickle(os.path.join(PATH, 'articubench/data/common_voice_geco_words_test_subset_slim_prot4.pkl'))
test_jahalt = pd.read_pickle(os.path.join(PATH, 'articubench/data/ja_halt.pickle'))


small_df_jahalt = pd.DataFrame()
small_df_jahalt['file'] = [str(i) + '_jahalt.pkl' for i in range(len(test_jahalt))]
small_df_jahalt['label'] = test_jahalt['Word']
small_df_jahalt['target_semantic_vector'] = test_jahalt['Word'].apply(lambda x: np.double(FASTTEXT_EMBEDDINGS.get_word_vector(x)))
small_df_jahalt['target_sig'] = test_jahalt['Signal']
small_df_jahalt['target_sr'] = test_jahalt['SampleRate'] #SampleRate is 22050 for ja_halt
small_df_jahalt['len_cp'] = test_jahalt.apply(lambda row: int(2 * math.ceil(44100 / 220 * len(row['Signal']) / row['SampleRate'])), axis=1) #TODO: Wielang sind die cps? Hab als quickfix einfach 402 * len(sig_rec) / sig_sr genommen und dann auf nächste even int geceilt
small_df_jahalt['reference_cp'] = None
small_df_jahalt['reference_tongue_height'] = None
small_df_jahalt['reference_ema'] = None #TODO: EMA Daten einfügen
small_df_jahalt['reference_ema_TT'] = None #TODO: EMA Daten einfügen
small_df_jahalt['reference_ema_TB'] = None #TODO: EMA Daten einfügen
test_jahalt.columns


WARNING! The *paule* package is still in alpha. To download pretrained weights for the PAULE model, which is defined in `paule.paule.Paule`, you can invoke `paule.util.download_pretrained_weights()`.
Version of the VocalTractLab library: "API 2.6.0quantling Dec 22 2022"


In [ ]:
small_df_geco = pd.DataFrame()
small_df_geco['file'] = test_small['file_name']
small_df_geco['label'] = test_small['label']
small_df_geco['target_semantic_vector'] = test_small['vector']
small_df_geco['target_sig'] = test_small['wav_rec']
small_df_geco['target_sr'] = test_small['sr_rec']
small_df_geco['len_cp'] = test_small['cp_norm'].apply(lambda x: len(x))
small_df_geco['reference_cp'] = test_small['cp_norm'].apply(lambda x: util.inv_normalize_cp(x)) #hier cp_norm oder inverses cp_norm?
small_df_geco['reference_tongue_height'] = None
small_df_geco['reference_ema'] = None

In [3]:
import fasttext.util
import numpy as np
import pandas as pd
#FASTTEXT_EMBEDDINGS = fasttext.load_model('/home/bob/AndreWorkland/articubench/cc.de.300.bin')

LABEL_VECTORS = pd.read_pickle('/home/bob/AndreWorkland/articubench/articubench/data/lexical_embedding_vectors.pkl')
# Create new entries for "ja" and "halt"
new_entries = pd.DataFrame({
    'label': ['ja', 'halt'],
    'vector': [np.round(FASTTEXT_EMBEDDINGS.get_word_vector('ja'),4), np.round(FASTTEXT_EMBEDDINGS.get_word_vector('halt'),4)],
    'phones': [['j', 'a'], ['h', 'a', 'l', 't']],
    'durations': [[0.13, 0.11], [0.0966, 0.063, 0.0766, 0.085]]
})

# Append the new entries to LABEL_VECTORS
LABEL_VECTORS = pd.concat([LABEL_VECTORS, new_entries], ignore_index=True)
LABEL_VECTORS.to_pickle('/home/bob/AndreWorkland/articubench/articubench/data/lexical_embedding_vectors2.pkl')

In [ ]:
from articubench.score import score
from articubench.control_models import synth_paule_fast, synth_baseline_schwa, synth_paule_acoustic_semvec, synth_baseline_segment
import pickle

def test_paule_fast():

    results = score(synth_baseline_segment, preloaded_data=small_df_jahalt[:10], tasks='all', subscores='all', return_individual_subscores=True
                    )
    with open('minimal_example_results_fast.pkl', 'wb') as pfile:
        pickle.dump((results), pfile)

test_paule_fast()

WARNING! The *articubench* package is still in alpha and the package   does not contain all the files you need to execute the functions defined here. To download model weights (around 50 MB) run `articubench.util.download_pretrained_weights()` once.
Version of the VocalTractLab library: "API 2.5.2quantling Apr 26 2022"
WARNING! The *paule* package is still in alpha. To download pretrained weights for the PAULE model, which is defined in `paule.paule.Paule`, you can invoke `paule.util.download_pretrained_weights()`.
Version of the VocalTractLab library: "API 2.6.0quantling Dec 22 2022"
pretrained_models exist already. Skip download. Path is /home/bob/miniconda3/lib/python3.12/site-packages/paule/pretrained_models
Version of pretrained weights is "common voice 20240306"
Version of pretrained weights is "common voice 20240306"


NameError: name 'small_df_jahalt' is not defined